In [ ]:
!python --version

In [ ]:
import os, sys
from scipy import stats
import numpy as np
import pandas as pd

from scipy import stats

is_colab = False

if is_colab:
    # create src and upload stat_lib into it
    from src import stat_lib
else:
    sys.path.insert(1, '../src/')
    import stat_lib

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

### Versão do SciPy

In [ ]:
import scipy

scipy.__version__

### stats.mannwhitneyu

O **teste U de Mann-Whitney** é um `teste não paramétrico` da hipótese nula de que a distribuição subjacente à amostra x é a mesma que a distribuição subjacente à amostra y. É frequentemente usado como um teste de diferença de localização entre distribuições.


**Dualidade**

O **teste U de Mann-Whitney** é o análogo não paramétrico é análogo ao **teste t para duas amostras independentes**.


https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html

In [ ]:
np.random.seed(42)

MU1=10; SSD1=2.5; N1=30
MU2=12; SSD2=2.5; N2=23

samp1 = np.random.normal(loc=MU1, scale=SSD1, size=N1)
samp2 = np.random.normal(loc=MU2, scale=SSD2, size=N2)

len(samp1), len(samp2)

## 📌 Estatística não-paramétrica

  - É baseada em ranqueamento (ranks)
  - Por exemplo:
    - Dada duas sequências de valores
    - Concatena-se ambas
    - Ordena-se
    - Obtém a ordem de cada valor das amostras concatendas e ordenadas


## 📌 Estatística e parâmetros

  - A estatística paramétrica é baseada em parâmetros como média, desvio padrão, graus de liberdade


  - A estatística não-paramétrica é baseada em ranqueamento (ranks)



## 📌 Comparação com estatística paramétrica t-test

  - t-test → usa valores reais
  - Mann–Whitney → usa ordenação (ranqueamento)

### 🧪 Teste t (independente)

Conceito:

  - Normalidade dos erros (diferença entre duas amostras, resíduos), assumes H0:

     - $\epsilon \sim \mathcal{N}(0, \sigma^2)$


  - Dadas duas amostras:
    - Observa-se as médias amostrais $\bar{X_1}$ e$\bar{X_2}$
    - Pergunta-se: será que μ₁ == μ₂?

Requisitos do teste-t

  - Dados das duas distribuiçoes:
    - Assume-se normalidade das populações (ou dos erros do modelo)
    - Já as amostras observadas não precisem apresentar aparência perfeitamente normal (o teste de Shapiro-Wilk consegue responder)
    - Assume variâncias próximas (ou usa Welch)
    - são iid
  - Compara-se médias populacionais (μ₁ == μ₂)
  - Estatística do teste segue distribuição t sob H₀
  - Usualmente para N amostra <= 30 ... mas pode ser utilizado para quaisquer N amostrais.

> 𝐻0: μ1 == μ2  
> 𝐻a: μ1 != μ2  


### 🧪 Mann–Whitney

Testa se os valores amostrais estão intercalados no ranqueamento de duas amostras ou não

  - Não presumo o tipo da distribuição dos valores amostrados
  - Não se preocupa com variâncias, sinais e outliers
  - Não utiliza de parâmetros como médias e desvios padrões
  - Somente utiliza o ranqueamento de valores concatenados e ordenados


O que conceito de **misturar** é didático e visual, na verdade se Mann-Whitney contabiliza os afastamento do ranqueamento de duas séries de valores concatenadas (ajuntar) e ordenadas.


> 𝐻0: os ranqueamentos se misturam (séries próximas)  

  - Grupo de dados semelhantes
  - Dado o ranqueamento A
    - o ranqueamento B se mistura ao ranqueamento A

> 𝐻a: os ranqueamentos estão afastados (baixa ou nenhuma sobreposição)  

  - Grupo de dados distintos
  - Dado o ranqueamento A
    - descolamento para direita: proximamente e/ou posterior ao rank A vem o B
    - descolamento para esquerda: proximamente e/ou anterior ao rank A vem o B


### Ranqueamento

Ranqueamento é substituir os valores amostrados por suas posições ordenadas.

In [ ]:
samp1

In [ ]:
samp1_ord = (samp1)
samp1_ord.sort()

# os 10 primeiros
samp1_ord[:5]

In [ ]:
samp1_rank = np.arange(1, len(samp1)+1)
samp1_rank[:5]

In [ ]:
for i in range(len(samp1_rank)):
    rank = i+1
    print(f"val {samp1_ord[i]:6.3f} -> rank {rank:2}")

#### 📌 Por que usar ranks?

Porque:
  - Há somente ordernação simples, não dependência da escala
  - Outliers entram na ordenação e não geram viés
  - Independe do tipo da distribuição (normal, simétrica, etc) - agora só há uma ordem
  - Compara-se ordens entre 2 grupos

#### 🧪 Exemplo 'simples'

samp1 = \[3, 8, 12]
samp2 = \[4, 9, 150]

##### Ordem geral

3, 4, 8, 9, 12 150
1, 2, 3, 4, 5, 6

outlier=150


--> ordem samp1: 1, 3, 5
--> ordem samp2: 2, 4, 6

#### Ranqueamento

Ra = 1+3+5 = 9

Ua = Ra - \[na (na + 1)] /2

In [ ]:
sampA = [1, 3, 5]
sampB =  [4, 9, 150]

sequencia = sampA + sampB
sequencia.sort()
sequencia

In [ ]:
# i+1 = rank
ranks_a = [i+1 for i in range(len(sequencia)) if sequencia[i] in sampA]
Ra = int(np.sum(ranks_a))

Ra, ranks_a

In [ ]:
na = len(sampA)
na

In [ ]:
Ua = Ra - na*(na+1)/2
Ua

#### 🧠 Interpretação intuitiva

Quantas vezes um valor de A é maior que um valor de B

```
1 > [4, 9, 150] -> 0
3 > [4, 9, 150] -> 0
5 > [4, 9, 150] -> 1

total (U) = 1
```

In [ ]:
stat, p = stats.mannwhitneyu(samp1, samp2, alternative='two-sided')

print("U =", stat)
print("p-value =", p)

In [ ]:
samples = [float(x) for x in samp1] +  [float(x) for x in samp2]
samples.sort()
samples[:10]

In [ ]:
rank = [i+1 for i in range(len(samples))]

In [ ]:
is_a = [samples[i] in samp1 for i in range(len(samples))]
is_b = [samples[i] in samp2 for i in range(len(samples))]

In [ ]:
is_a[:10]

In [ ]:
is_a[-10:]

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 7))

ax=axes[0]
val_rank_a = [1 if it_is else 0 for it_is in is_a]
ax = sns.barplot(x=rank, y=val_rank_a, color='navy', ax=ax)

labels = [rank[i] if is_a[i]==True else '' for i in range(len(rank))]
ax.set_xticks(range(len(rank)))
ax.set_xticklabels(labels, rotation=270);

title = "Conceito de 'miturar' ao se compara o rankeamento de duas amostras"
ax.set_title(title)

ax=axes[1]
val_rank_b = [1 if it_is else 0 for it_is in is_b]
ax = sns.barplot(x=rank, y=val_rank_b, color='darkred', ax=ax)

labels = [rank[i] if is_b[i]==True else '' for i in range(len(rank))]
ax.set_xticks(range(len(rank)))
ax.set_xticklabels(labels, rotation=270);